In [13]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
from shapely.geometry import Point, shape, mapping, Polygon, MultiPolygon
from geoband.API import *
import folium
import json
import geopandas as gpd

In [14]:
def generate_point(data, lat, lon, crs = "epsg:4326"):
    data = gpd.GeoDataFrame(
    data, crs = crs,
    geometry = [Point(latlon) for latlon in zip(data[lat],data[lon])])
    return data

In [15]:
data2 = gpd.read_file('2.오산시_어린이교통사고_격자.geojson')

In [16]:
parking = pd.read_excel("경기도_오산시_주차장.xlsx")
parking1 = pd.read_excel("오산시시설관리공단_주차장정보.xlsx")
parking = pd.concat([parking,parking1],axis=0).reset_index()
parking = parking.drop(columns = ["index"])

parking = generate_point(parking,"경도","위도")
park_buff = parking[["주차장명","경도","위도","geometry"]]
park_buff["geometry"] = parking.geometry.buffer(0.0004)

# data2 (어린이 교통사고 격자) 기준으로 유동인구 건수 count 
park_inter = gpd.overlay(data2,park_buff, how = "intersection")
park_gid = list(park_inter.gid.unique())

# 일부수정 
park_gid = [x for x in park_gid if x not in ["다사606088","다사605088","다사610079","다사609079"]]
park_gid += ["다사607088","다사608088","다사610078","다사609078","다사608078","다사608077","다사610077"]

park_yn =[]
for i in range(len(data2)):
    if data2["gid"][i] in park_gid: park_yn.append(1)
    else: park_yn.append(0)

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
/opt/app-root/lib/python3.6/site-packages/geopandas/geodataframe.py:1190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [17]:
final = pd.read_csv("final_all.txt", delimiter = "\t")

In [18]:
final.columns

Index(['gid', 'accident_cnt', 'geometry', 'schoolzone_num', 'safetysign_num',
       'bump_num', 'cctv_num', 'child_num', 'school_num', 'facility_num',
       'signal_num', 'busstop_num', 'crosswalk_num', 'sidewalk_num',
       'ill_park', 'foot_traf', 'car_num', 'node', 'park_yn', 'foot_traf_mean',
       'traffic', 'bigcar_ratio', 'rushhour_itn', 'rushhour_freq',
       'road_width'],
      dtype='object')

In [19]:
final = final.drop(columns = ["foot_traf","rushhour_itn"])

In [20]:
final = final[['gid', 'geometry','accident_cnt', 'schoolzone_num', 'safetysign_num',
       'bump_num', 'cctv_num', 'child_num', 'school_num', 'facility_num',
       'signal_num', 'busstop_num', 'crosswalk_num', 'sidewalk_num',
       'ill_park', 'car_num', 'node', 'park_yn', 'foot_traf_mean', 'traffic',
       'bigcar_ratio', 'rushhour_freq', 'road_width']]

In [21]:
final.isnull().sum()

gid                  0
geometry             0
accident_cnt         0
schoolzone_num       0
safetysign_num       0
bump_num             0
cctv_num             0
child_num            0
school_num           0
facility_num         0
signal_num           0
busstop_num          0
crosswalk_num        0
sidewalk_num         0
ill_park             0
car_num              0
node                 0
park_yn              0
foot_traf_mean       0
traffic           2820
bigcar_ratio      2820
rushhour_freq     2820
road_width        2820
dtype: int64

In [22]:
park_df = pd.DataFrame({"park_yn":park_yn})

In [23]:
park_df

,park_yn
0,0
1,0
2,0
3,0
4,0
...,...
4505,0
4506,0
4507,0
4508,0


In [25]:
sum(park_df["park_yn"]==1)

140

In [26]:
sum(park_df["park_yn"]==0)

4370

In [28]:
final[park_df["park_yn"]==1].isnull().sum()

gid                0
geometry           0
accident_cnt       0
schoolzone_num     0
safetysign_num     0
bump_num           0
cctv_num           0
child_num          0
school_num         0
facility_num       0
signal_num         0
busstop_num        0
crosswalk_num      0
sidewalk_num       0
ill_park           0
car_num            0
node               0
park_yn            0
foot_traf_mean     0
traffic           24
bigcar_ratio      24
rushhour_freq     24
road_width        24
dtype: int64

In [29]:
final.traffic.value_counts().sort_index()

99.810000       1
148.835000      1
160.985000      2
173.515000      1
226.560000      3
               ..
27828.650000    2
27943.905000    1
27981.321667    1
28000.030000    1
28116.530000    1
Name: traffic, Length: 1261, dtype: int64

In [30]:
final.bigcar_ratio.value_counts().sort_index()

0.006008    1
0.006683    1
0.006845    1
0.007378    1
0.007547    1
           ..
0.392140    1
0.433779    2
0.445333    1
0.485651    1
0.513795    1
Name: bigcar_ratio, Length: 1260, dtype: int64

In [31]:
final.rushhour_freq.value_counts().sort_index()

0.000000      2
0.025000      2
0.033333      1
0.068333      1
0.078000      1
             ..
99.770000     2
99.780000     2
99.900000     1
99.920000     2
100.000000    4
Name: rushhour_freq, Length: 1251, dtype: int64

In [32]:
final.road_width.value_counts().sort_index()

0.250000     89
0.277778      1
0.281250      2
0.300000      1
0.312500      5
           ... 
0.916667      8
0.937500      3
0.950000      1
0.958333      1
1.000000    141
Name: road_width, Length: 81, dtype: int64

In [33]:
for i in range(len(final)):
    if np.isnan(final.traffic[i]) and park_yn[i] == 1: 
        final.loc[i,"traffic"] = np.min(final.traffic)
        final.loc[i,"bigcar_ratio"] = np.min(final.bigcar_ratio)
        final.loc[i,"rushhour_freq"] = np.min(final.rushhour_freq)
        final.loc[i,"road_width"] = np.min(final.road_width)

In [34]:
final.isnull().sum()

gid                  0
geometry             0
accident_cnt         0
schoolzone_num       0
safetysign_num       0
bump_num             0
cctv_num             0
child_num            0
school_num           0
facility_num         0
signal_num           0
busstop_num          0
crosswalk_num        0
sidewalk_num         0
ill_park             0
car_num              0
node                 0
park_yn              0
foot_traf_mean       0
traffic           2796
bigcar_ratio      2796
rushhour_freq     2796
road_width        2796
dtype: int64

In [35]:
final_woNA = final[final.traffic.isnull()==False]

In [36]:
final_woNA.shape

(1714, 23)

In [37]:
final_woNA.columns

Index(['gid', 'geometry', 'accident_cnt', 'schoolzone_num', 'safetysign_num',
       'bump_num', 'cctv_num', 'child_num', 'school_num', 'facility_num',
       'signal_num', 'busstop_num', 'crosswalk_num', 'sidewalk_num',
       'ill_park', 'car_num', 'node', 'park_yn', 'foot_traf_mean', 'traffic',
       'bigcar_ratio', 'rushhour_freq', 'road_width'],
      dtype='object')

In [38]:
final_woNA.to_csv("final_woNA.csv",encoding='utf-8', sep='\t')